In [30]:
import random

# Define the initial puzzle state and the goal state
all_possible_elements = [1, 2, 3, 4, 5, 6, 7, 8, 0] #all possible elements of 8 puzzle.
initial_state = list(all_possible_elements)
random.shuffle(initial_state)
goal_state =[5, 0, 8, 4, 2, 1, 7, 3, 6]

# Define genetic algorithm parameters
population_size = 10
mutation_probability = 0.1
crossover_probability = 0.8
max_generations = 1000


# Define the fitness functions: misplaced tiles and Manhattan distance
def misplaced_tiles(state):
    displaced = 0
    for i in range(9):
        if state[i] != goal_state[i]:
            if  state[i]==0:
              continue
            displaced += 1
    return displaced

def manhattan_distance(state):
    distance = 0
    for i in range(9):
        if state[i] != goal_state[i]:
            if  state[i]==0:
              continue
            distance += abs(i//3 - goal_state.index(state[i])//3) + abs(i%3 - goal_state.index(state[i])%3)
    return distance


 # Function to create an initial population of random puzzle states
def create_initial_population():
    # print([random.sample(range(9), 9) for _ in range(population_size)])
    return [random.sample(initial_state, 9) for _ in range(population_size)]

# Function to perform roulette wheel selection
def roulette_wheel_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probabilities = [score / total_fitness for score in fitness_scores]
    return random.choices(population, weights=selection_probabilities, k=2)

# Function to perform one-point crossover
def one_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, 7)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Function to perform mutation
def mutation(state):
    if random.random() < mutation_probability:
        i, j = random.sample(range(9), 2)
        state[i], state[j] = state[j], state[i]
    return state



# Genetic algorithm
population = create_initial_population()
generation = 0

while generation < max_generations:
    fitness_scores = [manhattan_distance(state) for state in population]
    best_fitness = min(fitness_scores)
    best_solution = population[fitness_scores.index(best_fitness)]
    # print(best_solution)
    if best_fitness == 0:
        print("Solution found!")
        print("Best solution:", best_solution)
        break

    new_population = []

    for _ in range(population_size // 2):
        parent1, parent2 = roulette_wheel_selection(population, fitness_scores)

        if random.random() < crossover_probability:
            child1, child2 = one_point_crossover(parent1, parent2)
        else:
            child1, child2 = parent1, parent2

        child1 = mutation(child1)
        child2 = mutation(child2)

        new_population.extend([child1, child2])

    population = new_population
    generation += 1

if generation == max_generations:
    print("Maximum generations reached. No solution found.")


Maximum generations reached. No solution found.


In [ ]:
import random
import math
import time

# Define the representation of a puzzle state
class PuzzleState:
    def __init__(self, state):
        self.state = state
        self.fitness = None

# Define the initial population
def generate_initial_population(population_size, goal_state):
    initial_population = []
    for _ in range(population_size):
        random.shuffle(goal_state)
        initial_population.append(PuzzleState(list(goal_state)))
    return initial_population

# Define the cooling schedule and acceptance probability function for Simulated Annealing
def cooling_schedule(iteration):
    return 1 / (1 + math.log(1 + iteration))

def acceptance_probability(old_cost, new_cost, temperature):
    return math.exp((old_cost - new_cost) / temperature)

# Implement the simulated annealing algorithm
def simulated_annealing(start_state, goal_state, fitness_function):
    current_state = PuzzleState(start_state)
    temperature = 1.0
    iterations = 0

    while temperature > 0.1:
        iterations += 1
        neighbors = generate_neighbors(current_state.state)
        neighbor = random.choice(neighbors)
        current_cost = fitness_function(current_state.state, goal_state)
        neighbor_cost = fitness_function(neighbor, goal_state)

        if neighbor_cost < current_cost or random.random() < acceptance_probability(current_cost, neighbor_cost, temperature):
            current_state = PuzzleState(neighbor)

        temperature *= cooling_schedule(iterations)

    return current_state, iterations


def generate_neighbors(state):
    neighbors = []
    blank_row, blank_col = None, None

    # Find the position of the blank tile
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == "B":
                blank_row, blank_col = i, j
                break

    # Generate neighboring states by moving the blank tile
    for i, j in [(blank_row-1, blank_col), (blank_row+1, blank_col), (blank_row, blank_col-1), (blank_row, blank_col+1)]:
        if 0 <= i < len(state) and 0 <= j < len(state[i]):
            new_state = [row[:] for row in state]  # Create a copy of the state
            new_state[blank_row][blank_col], new_state[i][j] = new_state[i][j], new_state[blank_row][blank_col]  # Swap tiles
            neighbors.append(new_state)

    return neighbors



# Define selection (Roulette Wheel Selection)
def roulette_wheel_selection(population):
    total_fitness = sum([1/state.fitness for state in population])
    probabilities = [1/(state.fitness*total_fitness) for state in population]
    return random.choices(population, weights=probabilities)[0]

# Define crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1.state)-1)
    child_state = parent1.state[:crossover_point] + parent2.state[crossover_point:]
    return PuzzleState(child_state)

# Define mutation
def mutate(state):
    mutated_state = state.state.copy()
    index1, index2 = random.sample(range(len(mutated_state)), 2)
    mutated_state[index1], mutated_state[index2] = mutated_state[index2], mutated_state[index1]
    return PuzzleState(mutated_state)

# Implement the genetic algorithm
def genetic_algorithm(start_state, goal_state, population_size, mutation_prob, crossover_prob, fitness_function):
    population = generate_initial_population(population_size, goal_state)
    generations = 0

    while generation < 10000:
        for state in population:
            state.fitness = fitness_function(state.state, goal_state)

        best_state = min(population, key=lambda x: x.fitness)
        if best_state.fitness == 0:
            return best_state, generations

        new_population = [best_state]

        while len(new_population) < population_size:
            parent1 = roulette_wheel_selection(population)
            parent2 = roulette_wheel_selection(population)

            if random.random() < crossover_prob:
                child = crossover(parent1, parent2)
                if random.random() < mutation_prob:
                    child = mutate(child)
                new_population.append(child)

        population = new_population
        generations += 1
    if generation == max_generations:
        return("Maximum generations reached. No solution found.")

# Execute the algorithm and record the required information
start_state = [[5, "B", 8], [4, 2, 1], [7, 3, 6]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 'B']]
population_size = 10
mutation_prob = 0.2
crossover_prob = 0.8

start_time_genetic = time.time()  # Record the start time
# result_genetic, generations = genetic_algorithm(start_state, goal_state, population_size, mutation_prob, crossover_prob, misplaced_tiles)
result_genetic, generations = genetic_algorithm(start_state, goal_state, population_size, mutation_prob, crossover_prob, manhattan_distance)
end_time_genetic = time.time()  # Record the end time

# Calculate the time taken for execution
execution_time_genetic = end_time_genetic - start_time_genetic

# Execute the simulated annealing algorithm
start_time_annealing = time.time()  # Record the start time
# result_annealing, iterations = simulated_annealing(start_state, goal_state, misplaced_tiles)
result_annealing, iterations = simulated_annealing(start_state, goal_state, manhattan_distance)
end_time_annealing = time.time()  # Record the end time

# Calculate the time taken for execution
execution_time_annealing = end_time_annealing - start_time_annealing

# Print out the required information for Genetic Algorithm
if result_genetic.fitness == 0:
    print("Genetic Algorithm Success Message")
    print("Start State:", start_state)
    print("Goal State:", goal_state)
    print("Total number of states explored:", generations * population_size)
    print("Total number of states to the optimal path:", len(result_genetic.state))
    print("Optimal Path:", result_genetic.state)
    print("Optimal Path Cost:", result_genetic.fitness)
    print(f"Time taken for execution: {execution_time_genetic:.4f} seconds")  # Print the execution time
else:
    print("Genetic Algorithm Failure Message")
    print("Start State:", start_state)
    print("Goal State:", goal_state)
    print("Total number of states explored before termination:", generations * population_size)

# Print out the required information for Simulated Annealing
if result_annealing.fitness == 0:
    print("Simulated Annealing Success Message")
    print("Start State:", start_state)
    print("Goal State:", goal_state)
    print("Total number of states explored:", iterations)
    print("Total number of states to the optimal path:", len(result_annealing.state))
    print("Optimal Path:", result_annealing.state)
    print("Optimal Path Cost:", result_annealing.fitness)
    print(f"Time taken for execution: {execution_time_annealing:.4f} seconds")  # Print the execution time
else:
    print("Simulated Annealing Failure Message")
    print("Start State:", start_state)
    print("Goal State:", goal_state)
    print("Total number of states explored before termination:", iterations)  # Use 'iterations' here